## Comparing RAG vs SFT on a Synthetic Biological Dataset

- Karthik Manishankar (karthik.manishankar@sjsu.edu)
- Anwar Mujeeb (muhammedanwar.mujeebputhenveedu@sjsu.edu)

### Final Project:
- The Model we chose for this assignment was the Pythia - 410M model. In this project, we compare and evaluate Bleu/Rouge scores for a model trained on biology question data: https://www.kaggle.com/datasets/thedevastator/synbio-problem-solution-dataset

- Objective:
To enhance the accuracy and efficiency of problem-solving in synthetic biology by implementing a RAG model and a SFT model and comparing their responses.

- Problem Complexity:
Traditional models w/o a retriever or knowledge of a specialized domain may not adequately capture breadth of information necessary to solve such problems

- Slides link: https://docs.google.com/presentation/d/1JK7RCoR-Y1K8rXa9I6lHMnEE_9EcAIo4emaUtH05uqU/edit?usp=sharing




# Intro Imports


In [ ]:
!pip install huggingface_hub
!pip install transformers
!pip install -q torch transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap
!pip install datasets
!pip install datasets-faiss
!pip install langchain
!pip install chromadb
!pip install langchain-community
!pip install langchain-text-splitters
!pip install langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 61.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15

In [ ]:
from google.colab import files
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceEndpoint


# Retrieval Augmented Generation Pythia



## Read and View Data


In [ ]:
df = pd.read_csv('train 2.csv')
print(df.head())

                         role_1    topic;                      sub_topic  \
0  Biologist_RoleType.ASSISTANT  Taxonomy  Identification of new species   
1  Biologist_RoleType.ASSISTANT  Taxonomy  Identification of new species   
2  Biologist_RoleType.ASSISTANT  Taxonomy  Identification of new species   
3  Biologist_RoleType.ASSISTANT  Taxonomy  Identification of new species   
4  Biologist_RoleType.ASSISTANT  Taxonomy  Identification of new species   

                                           message_1  \
0  What are the common techniques used in identif...   
1  How can DNA sequencing be used to identify new...   
2  "How can you identify and classify a newly dis...   
3  Identify the key differences between two speci...   
4  How can DNA analysis be used in the identifica...   

                                           message_2  
0  Identifying a new species and categorizing it ...  
1  DNA sequencing can be used to identify new spe...  
2  To identify and classify a newly disco

In [ ]:
df = pd.read_csv('train 2.csv')
df = df.filter(['message_2'])
df.head()

,message_2
0,Identifying a new species and categorizing it ...
1,DNA sequencing can be used to identify new spe...
2,To identify and classify a newly discovered sp...
3,To identify the key differences between two sp...
4,DNA analysis can be used in the identification...


In [ ]:
df

,message_2
0,Identifying a new species and categorizing it ...
1,DNA sequencing can be used to identify new spe...
2,To identify and classify a newly discovered sp...
3,To identify the key differences between two sp...
4,DNA analysis can be used in the identification...
...,...
19995,Current conservation efforts and methods being...
19996,The conservation and preservation of endangere...
19997,To effectively preserve and conserve endangere...
19998,The preservation of endangered fungal species ...


## Load data into Document format for Langchain text splitter and vectordb

In [ ]:
knowledge = "train.csv"
column = "message_2"
loader = DataFrameLoader(df,page_content_column='message_2')
data = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
pages = text_splitter.split_documents(data)


In [ ]:
pages[0]

Document(page_content="Identifying a new species and categorizing it within the existing taxonomy system involves several common techniques. These techniques can be broadly classified into morphological, molecular, and ecological approaches. Here's an overview of these techniques:\n\n1. Morphological analysis: This is the traditional method of identifying a new species based on its physical characteristics. Scientists examine the external and internal features of an organism, such as its size, shape, color, and structure. They also study the anatomy and physiology of the organism, including its skeletal, muscular, and reproductive systems. By comparing these features with those of known species, scientists can determine if the organism represents a new species.")

## Initialize model
 - Setup variables for model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m-v0")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-410m-v0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L12-v2',     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)
print(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L12-v2' cache_folder=None model_kwargs={'device': 'cuda'} encode_kwargs={'normalize_embeddings': False} multi_process=False show_progress=False


In [ ]:
text = "Testing Document" #Test embeddings
query_result = embeddings.embed_query(text)
print(query_result)

[-0.04857775568962097, 0.09214870631694794, -0.062493715435266495, 0.023279283195734024, -0.014584433287382126, 0.026638561859726906, -0.04024218022823334, 0.06627607345581055, -0.11924005299806595, 0.007337752263993025, 0.0014877539360895753, 0.002070985035970807, 0.02933715283870697, -0.04662419110536575, -0.08670321106910706, -0.09233732521533966, 0.05187313258647919, -0.043928854167461395, 0.029253683984279633, 0.0648825392127037, 0.002991404850035906, 0.0019400097662582994, 0.04574955254793167, -0.004562943708151579, -0.10325367003679276, 0.05049062892794609, -0.04005509987473488, -0.026290765032172203, -0.028512705117464066, -0.09092868119478226, -0.0021027743350714445, 0.029052572324872017, 0.02665633335709572, 0.04981369525194168, 0.06361478567123413, -0.04897451773285866, -0.006610200274735689, -0.0757475346326828, 0.07299578189849854, 0.001604618621058762, 0.05049126222729683, -0.041425738483667374, -0.019700469449162483, 0.030646292492747307, 0.005448825657367706, 0.00540514

In [ ]:
db = Chroma.from_documents(pages, embeddings, persist_directory="./chroma_db") #setup vectordb and persist db in case of runtime loss.

In [ ]:
searchDocs = db.similarity_search("What is DNA made up of?")
print(searchDocs[0].page_content)

The structure of DNA plays a crucial role in its replication, ensuring that genetic information is accurately copied and passed on to new cells. DNA (deoxyribonucleic acid) is a double-stranded molecule composed of two complementary strands that run in opposite directions, with each strand made up of nucleotides. These nucleotides consist of a sugar molecule (deoxyribose), a phosphate group, and one of four nitrogenous bases: adenine (A), cytosine (C), guanine (G), and thymine (T). The two strands are held together by hydrogen bonds between the nitrogenous bases, forming base pairs (A-T and C-G).

There are several ways in which the structure of DNA contributes to its replication:


## Setup RAG

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
repo_id = "EleutherAI/pythia-410m"

pipe = pipeline("text-generation", model="EleutherAI/pythia-410m",
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.5, "max_length": 512},
)

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 5})
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [ ]:
rag_chain.invoke("What is DNA made up of and what are some differences with ancient DNA?")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is DNA made up of and what are some differences with ancient DNA? \nContext: The structure of DNA plays a crucial role in its replication, ensuring that genetic information is accurately copied and passed on to new cells. DNA (deoxyribonucleic acid) is a double-stranded molecule composed of two complementary strands that run in opposite directions, with each strand made up of nucleotides. These nucleotides consist of a sugar molecule (deoxyribose), a phosphate group, and one of four nitrogenous bases: adenine (A), cytosine (C), guanine (G), and thymine (T). The two strands are held together by hydrogen bonds between the nitrogenous bases, forming base pairs (A-T and C-G).\n\nThere are several ways in which the structure of DNA c

In [ ]:
def rag_process(question):
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
    )
    result = rag_chain.invoke(question)
    parts = result.split("\nAnswer:")
    # answer = result.split("Answer:\n\nA:\n\n")[-1].strip()
    print(parts[1])
    return parts[1]

## Setup new df for **RAG**

In [ ]:
df1 = pd.read_csv('train 2.csv')
df_600 = df1.head(600)
print(df_600)

                           role_1    topic;  \
0    Biologist_RoleType.ASSISTANT  Taxonomy   
1    Biologist_RoleType.ASSISTANT  Taxonomy   
2    Biologist_RoleType.ASSISTANT  Taxonomy   
3    Biologist_RoleType.ASSISTANT  Taxonomy   
4    Biologist_RoleType.ASSISTANT  Taxonomy   
..                            ...       ...   
595  Biologist_RoleType.ASSISTANT  Taxonomy   
596  Biologist_RoleType.ASSISTANT  Taxonomy   
597  Biologist_RoleType.ASSISTANT  Taxonomy   
598  Biologist_RoleType.ASSISTANT  Taxonomy   
599  Biologist_RoleType.ASSISTANT  Taxonomy   

                                             sub_topic  \
0                        Identification of new species   
1                        Identification of new species   
2                        Identification of new species   
3                        Identification of new species   
4                        Identification of new species   
..                                                 ...   
595  Identification of the co

### Parse through each question in CamelAI csv and use that as question for RAG


In [ ]:
answerResults = []
count = 0
for question in df_600['message_1']:
    # print(question)
    count += 1
    print(count)
    result = rag_process(question)
    answerResults.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


1


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 Yes!

3. Using advanced statistical tools: In addition to morphological and anatomical analyses, researchers can use advanced statistical tools, such as principal component analysis (PCA), discriminant function analysis (DFA), and cluster analysis, to identify and classify new species. These tools can be applied to both morphological and anatomical analyses.

4. Combining morphological and anatomical analyses: Combining morphological and anatomical analyses can improve the accuracy of the identification of new species. For example, combining morphological and anatomical analyses can help distinguish between two closely related species.

5. Combining molecular and morphological analyses: Combining molecular and morphological analyses can help in the accurate classification of new species. For example, combining molecular and morphological analyses can help differentiate between two closely related species.

6. Combining phylogenetic and morphological analyses: Combining phylogenetic an

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 The correct order for the species Felis catus (domestic cat) is Carnivora.

A:

I think this is a good example of how to use the context in your question.  You have two questions, one about the order of the species, and another about the order of the genus.  In the first question, you want to find out which order the species belong to.  In the second question, you want to find out which genus the species belong to.  So, you need to figure out which order the species belong to.
First, you can use the context to help you determine which order the species belong to.  For instance, if you wanted to find out which genus the species belonged to, you could use the context to tell you which genus the species belongs to.  If you were looking at the order of the genus, you would use the context to tell you which genus the species belongs to.
Next, you can use the context to help you determine which order the species belong to.  For instance, if you wanted to find out which order the species bel

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 The correct order for the species Felis catus (domestic cat) according to the Linnaean classification system is as follows:

1. Kingdom: Animalia (Animals)
2. Phylum: Chordata (Chordates)
3. Class: Mammalia (Mammals)
4. Order: Carnivora (Carnivores)
5. Family: Felidae (Cats)
6. Genus: Felis
7. Species: Felis catus

The correct order of the species "Felis catus" (domestic cat) according to the Linnaean classification system is as follows:

1. Kingdom: Animalia (Animals)
2. Phylum: Chordata (Chordates)
3. Class: Mammalia (Mammals)
4. Order: Carnivora (Carnivores)
5. Family: Felidae (Cats)
6. Genus: Felis
7. Species: Felis catus

The correct order for the species Felis catus (domestic cat) according to the Linnaean classification system is as follows:

1. Kingdom: Animalia (Animals)
2. Phylum: Chordata (Chordates)
3. Class: Mammalia (Mammals)
4. Order: Carnivora (Carnivores)
5. Family: Felidae (Cats)
6. Genus: Felis
7. Species: Felis catus

The correct order for the species Felis catus (

KeyboardInterrupt: 

In [ ]:
otherResults = answerResults #answers

In [ ]:
print(otherResults) #view rag answers

[' Yes!\n\n3. Using advanced statistical tools: In addition to morphological and anatomical analyses, researchers can use advanced statistical tools, such as principal component analysis (PCA), discriminant function analysis (DFA), and cluster analysis, to identify and classify new species. These tools can be applied to both morphological and anatomical analyses.\n\n4. Combining morphological and anatomical analyses: Combining morphological and anatomical analyses can improve the accuracy of the identification of new species. For example, combining morphological and anatomical analyses can help distinguish between two closely related species.\n\n5. Combining molecular and morphological analyses: Combining molecular and morphological analyses can help in the accurate classification of new species. For example, combining molecular and morphological analyses can help differentiate between two closely related species.\n\n6. Combining phylogenetic and morphological analyses: Combining phylo

##Setup new dataframe with RAG Responses
 - Makes it easy to compare RAG responses with GPT

In [ ]:
df1 = pd.read_csv('train 2.csv')
df_200 = df1.head(200)
print(df_200)

                           role_1    topic;  \
0    Biologist_RoleType.ASSISTANT  Taxonomy   
1    Biologist_RoleType.ASSISTANT  Taxonomy   
2    Biologist_RoleType.ASSISTANT  Taxonomy   
3    Biologist_RoleType.ASSISTANT  Taxonomy   
4    Biologist_RoleType.ASSISTANT  Taxonomy   
..                            ...       ...   
195  Biologist_RoleType.ASSISTANT  Taxonomy   
196  Biologist_RoleType.ASSISTANT  Taxonomy   
197  Biologist_RoleType.ASSISTANT  Taxonomy   
198  Biologist_RoleType.ASSISTANT  Taxonomy   
199  Biologist_RoleType.ASSISTANT  Taxonomy   

                                             sub_topic  \
0                        Identification of new species   
1                        Identification of new species   
2                        Identification of new species   
3                        Identification of new species   
4                        Identification of new species   
..                                                 ...   
195  Identification of the co

In [ ]:
df_200['rag_results'] = pd.NA

<ipython-input-61-2b132bee7bf8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_200['rag_results'] = pd.NA


In [ ]:
print(df_200)

                           role_1    topic;  \
0    Biologist_RoleType.ASSISTANT  Taxonomy   
1    Biologist_RoleType.ASSISTANT  Taxonomy   
2    Biologist_RoleType.ASSISTANT  Taxonomy   
3    Biologist_RoleType.ASSISTANT  Taxonomy   
4    Biologist_RoleType.ASSISTANT  Taxonomy   
..                            ...       ...   
195  Biologist_RoleType.ASSISTANT  Taxonomy   
196  Biologist_RoleType.ASSISTANT  Taxonomy   
197  Biologist_RoleType.ASSISTANT  Taxonomy   
198  Biologist_RoleType.ASSISTANT  Taxonomy   
199  Biologist_RoleType.ASSISTANT  Taxonomy   

                                             sub_topic  \
0                        Identification of new species   
1                        Identification of new species   
2                        Identification of new species   
3                        Identification of new species   
4                        Identification of new species   
..                                                 ...   
195  Identification of the co

In [ ]:
df_200['rag_results'] = otherResults[:len(df_200)]
df_200.to_csv('updated_with_rag_200.csv', index=False)

<ipython-input-64-e04e1d09352a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_200['rag_results'] = otherResults[:len(df_200)]


In [ ]:
print(otherResults)

[' Yes!\n\n3. Using advanced statistical tools: In addition to morphological and anatomical analyses, researchers can use advanced statistical tools, such as principal component analysis (PCA), discriminant function analysis (DFA), and cluster analysis, to identify and classify new species. These tools can be applied to both morphological and anatomical analyses.\n\n4. Combining morphological and anatomical analyses: Combining morphological and anatomical analyses can improve the accuracy of the identification of new species. For example, combining morphological and anatomical analyses can help distinguish between two closely related species.\n\n5. Combining molecular and morphological analyses: Combining molecular and morphological analyses can help in the accurate classification of new species. For example, combining molecular and morphological analyses can help differentiate between two closely related species.\n\n6. Combining phylogenetic and morphological analyses: Combining phylo

In [ ]:
df_200

,role_1,topic;,sub_topic,message_1,message_2,rag_results
0,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,What are the common techniques used in identif...,Identifying a new species and categorizing it ...,Yes!\n\n3. Using advanced statistical tools: ...
1,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,How can DNA sequencing be used to identify new...,DNA sequencing can be used to identify new spe...,DNA sequencing can be used to identify and cl...
2,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,"""How can you identify and classify a newly dis...",To identify and classify a newly discovered sp...,A\n\nA:\n\nThe first step is to find out what...
3,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,Identify the key differences between two speci...,To identify the key differences between two sp...,\n\nA:\n\nThe first step is to identify the ke...
4,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,How can DNA analysis be used in the identifica...,DNA analysis can be used in the identification...,A new species is defined when there is a clea...
...,...,...,...,...,...,...
195,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of the correct order for a give...,Identify the correct order for the following s...,The correct order for the following species is...,1. Felis catus (domestic cat)\n\n1. Felis cat...
196,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of the correct order for a give...,What is the correct order for a species named ...,The correct order for the species Ursus americ...,The correct order for the species Ursus arcto...
197,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of the correct order for a give...,What is the correct order for a given species ...,The correct order for a given species belongin...,The correct order for the species Homo sapien...
198,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of the correct order for a give...,Which of the following orders does the species...,"The species Ursus maritimus, commonly known as...","Ursus arctos belongs to the family Ursidae, w..."


## Evaluate RAG
 - Retrieve ROUGE and BLEU scores


In [ ]:
from sklearn.model_selection import train_test_split

train_data, remaining = train_test_split(df_200, test_size=0.30, random_state=42)

validation_data, test_data = train_test_split(remaining, test_size=0.50, random_state=42)

print(f"Training size: {len(train_data)}, Validation size: {len(validation_data)}, Test size: {len(test_data)}")

Training size: 140, Validation size: 30, Test size: 30


In [ ]:
!pip install nltk rouge-score
import nltk
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

nltk.download('punkt')


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=2c55cacb9fa18fccb3956dcddb6bdf80a463aa4c8b7222df89a2667baaa03593
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import torch
import numpy as np
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m-v0")
def truncate_to_max_length(text, max_length=512):
    """ Truncate text to a maximum length in tokens. """
    tokens = tokenizer.tokenize(text)
    truncated_tokens = tokens[:max_length]  # Limit to max_length tokens
    return tokenizer.convert_tokens_to_string(truncated_tokens)
def evaluate_models(test_data, model_responses, reference_key='message_2', max_length=512):
    """ Evaluate model generated responses against truncated reference responses. """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    results = {}

    for model_name, responses in model_responses.items():
        print(f"Evaluating {model_name}...")
        references = [truncate_to_max_length(ref, max_length) for ref in test_data[reference_key].tolist()]
        truncated_responses = [truncate_to_max_length(resp, max_length) for resp in responses]

        bleu_score = corpus_bleu([[ref.split()] for ref in references], [resp.split() for resp in truncated_responses])
        rouge_scores = [scorer.score(ref, resp) for ref, resp in zip(references, truncated_responses)]

        avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
        avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

        results[model_name] = {
            'BLEU': bleu_score,
            'ROUGE-1': avg_rouge1,
            'ROUGE-L': avg_rougeL
        }

    return results

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(test_data.rag_results)

95      A sample of the species' DNA should be collec...
115     The most important feature of a living organi...
135    \n\nThe correct taxonomic authority for the sp...
195     1. Felis catus (domestic cat)\n\n1. Felis cat...
78      Subspecies are geographically isolated popula...
117    \n\nA:\n\nThe first thing I would do is look a...
75      A subspecies of a bird species may have diffe...
143     The correct genus for the species Canis lupus...
165     Ursus arctos belongs to the family Ursidae in...
98      There are many examples of convergent evoluti...
29     \n\nA:\n\nThe first sentence is correct.  You ...
114     The most reliable method for determining the ...
18      The specimen examined was not a new species; ...
158     The correct taxonomic rank for the species Pa...
5       The first characteristic is the size of the o...
152     The species "Canis lupus" belongs to the genu...
174    \n\nA:\n\nI think this is a good example of ho...
69      A subspecies is a group

In [ ]:
model_responses = {
    # 'Pythia_LLM': pythia_responses
    'RAG_Model': test_data.rag_results
}

evaluation_results = evaluate_models(test_data, model_responses)
print(evaluation_results)


Evaluating RAG_Model...
{'RAG_Model': {'BLEU': 0.07736538064526675, 'ROUGE-1': 0.23904315756400152, 'ROUGE-L': 0.16030767668663282}}


# Fine-tuned Pythia


##Dataset Preparation


In [ ]:
!pip install langchainhub

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

## Loading Dataset

* wait for the entire file to load before running this

In [ ]:
df = pd.read_csv('train.csv')
df.head()

,role_1,topic;,sub_topic,message_1,message_2
0,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,What are the common techniques used in identif...,Identifying a new species and categorizing it ...
1,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,How can DNA sequencing be used to identify new...,DNA sequencing can be used to identify new spe...
2,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,"""How can you identify and classify a newly dis...",To identify and classify a newly discovered sp...
3,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,Identify the key differences between two speci...,To identify the key differences between two sp...
4,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of new species,How can DNA analysis be used in the identifica...,DNA analysis can be used in the identification...


### Cleaning Dataset
* Handling Missing Values and Formatting

In [ ]:
# Check for missing values in each column
print(df.isnull().sum())

# Drop rows where any of the essential text fields are missing
df.dropna(subset=['message_1', 'message_2'], inplace=True)

# Optional: Fill missing categorical data or handle it as a separate category
df['role_1'].fillna('Unknown', inplace=True)
df['sub_topic'].fillna('General', inplace=True)


role_1       0
topic;       0
sub_topic    0
message_1    0
message_2    0
dtype: int64


### Data Splitting

In [ ]:
# Split data into training and remaining data
train_data, remaining = train_test_split(df, test_size=0.30, random_state=42)

# Split remaining data into validation and test sets
validation_data, test_data = train_test_split(remaining, test_size=0.50, random_state=42)

print(f"Training size: {len(train_data)}, Validation size: {len(validation_data)}, Test size: {len(test_data)}")


Training size: 14000, Validation size: 3000, Test size: 3000


## Preprocessing Dataset

### Tokenization
* Implement using ```tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m-v0")```

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m-v0")

# Check if the tokenizer has a pad token
if tokenizer.pad_token is None:
    print("Setting pad_token.")
    tokenizer.pad_token = tokenizer.eos_token  # Often, the EOS token can be used as a padding token.

# Now try tokenizing again
sample_text = train_data['message_1'].iloc[0]
encoded_text = tokenizer(sample_text, truncation=True, max_length=512, padding="max_length")
print(encoded_text)


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setting pad_token.
{'input_ids': [2347, 1057, 253, 3361, 273, 271, 4465, 1594, 15406, 275, 4292, 342, 611, 1282, 28353, 350, 9237, 2818, 616, 37085, 6654, 285, 5615, 2440, 13, 285, 752, 403, 253, 2330, 3520, 285, 9699, 5917, 273, 253, 1617, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Input Formatting

In [ ]:
def preprocess_data(data):
    """ Tokenizes and formats the dataset. """
    # Convert text data into input IDs and attention masks for the model
    tokens = data.apply(lambda row: tokenizer(row['message_1'] + " " + row['message_2'], truncation=True, max_length=512, padding="max_length"), axis=1)
    return pd.DataFrame.from_records(tokens.tolist())

# Apply preprocessing to each dataset part
train_tokens = preprocess_data(train_data)
validation_tokens = preprocess_data(validation_data)
test_tokens = preprocess_data(test_data)


In [ ]:
# Function to display tokenized data samples
def display_token_samples(tokenized_data, num_samples=3):
    print("Displaying tokenized data samples:")
    for index in range(num_samples):
        sample = tokenized_data.iloc[index]
        tokens = tokenizer.convert_ids_to_tokens(sample['input_ids'])
        print(f"Sample {index + 1}:")
        print("Tokens:", tokens)
        print("Token IDs:", sample['input_ids'])
        print("Attention Mask:", sample['attention_mask'])
        print("-----")

# Display samples from each dataset part
# The G token represents a space token which helps model distinguish prefixes and standalone words
display_token_samples(train_tokens)
display_token_samples(validation_tokens)
display_token_samples(test_tokens)

Displaying tokenized data samples:
Sample 1:
Tokens: ['How', 'Ġdoes', 'Ġthe', 'Ġpresence', 'Ġof', 'Ġan', 'Ġextra', 'ĠX', 'Ġchromosome', 'Ġin', 'Ġindividuals', 'Ġwith', 'ĠK', 'line', 'fel', 'ter', 'Ġsyndrome', 'Ġaffect', 'Ġtheir', 'Ġhormonal', 'Ġbalance', 'Ġand', 'Ġsexual', 'Ġdevelopment', ',', 'Ġand', 'Ġwhat', 'Ġare', 'Ġthe', 'Ġassociated', 'Ġphysical', 'Ġand', 'Ġcognitive', 'Ġsymptoms', 'Ġof', 'Ġthe', 'Ġcondition', '?', 'ĠK', 'line', 'fel', 'ter', 'Ġsyndrome', 'Ġis', 'Ġa', 'Ġgenetic', 'Ġcondition', 'Ġthat', 'Ġaffects', 'Ġmales', ',', 'Ġresulting', 'Ġfrom', 'Ġthe', 'Ġpresence', 'Ġof', 'Ġan', 'Ġextra', 'ĠX', 'Ġchromosome', '.', 'ĠInstead', 'Ġof', 'Ġthe', 'Ġtypical', 'ĠXY', 'Ġsex', 'Ġchromosomes', ',', 'Ġindividuals', 'Ġwith', 'ĠK', 'line', 'fel', 'ter', 'Ġsyndrome', 'Ġhave', 'ĠXX', 'Y', 'Ġchromosomes', '.', 'ĠThis', 'Ġextra', 'ĠX', 'Ġchromosome', 'Ġaffects', 'Ġhormonal', 'Ġbalance', 'Ġand', 'Ġsexual', 'Ġdevelopment', ',', 'Ġleading', 'Ġto', 'Ġvarious', 'Ġphysical', ',', 'Ġcognitive', ',

## Loading Model and Configuring Environment

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Loading the pythia model
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-410m-v0")

# Ensure the model is moved to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

## Model Fine-Tuning

### Optimizer

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_scheduler

num_epochs = 10

num_training_steps = len(train_tokens) * num_epochs  # define num_epochs as per your training plan
lr_scheduler = get_scheduler(
    name="linear",  #  also use 'cosine' or other schedules
    optimizer=optimizer,
    num_warmup_steps=0,  # or some fraction of num_training_steps
    num_training_steps=num_training_steps
)

### Implementing Batch Processing
* Making a more memory efficient dataset by storing tokens in memory only when called

In [ ]:
from torch.utils.data import DataLoader, Dataset

class SyntheticBiologyDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        tokens = self.tokenizer(text=row['message_1'] + " " + row['message_2'],
                                padding='max_length',
                                truncation=True,
                                max_length=self.max_length,
                                return_tensors="pt")
        return {key: val.squeeze() for key, val in tokens.items()}

# Create dataset objects
train_dataset = SyntheticBiologyDataset(train_data, tokenizer)
validation_dataset = SyntheticBiologyDataset(validation_data, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)  # You can adjust batch_size depending on your GPU capacity
validation_loader = DataLoader(validation_dataset, batch_size=4, shuffle=False, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import os

checkpoint_dir = "./model_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

def save_checkpoint(model, optimizer, filename="checkpoint.pth.tar"):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(checkpoint, os.path.join(checkpoint_dir, filename))


### Freezing all layers except for the last two layers
* Ensuring that model is not overfit

In [ ]:
num_trainable_layers = 2  # Number of layers to fine-tune

# Freeze all layers except the last num_trainable_layers
for name, param in model.named_parameters():
    # The layer numbers are embedded in the parameter names for transformers models
    if 'transformer.h.' in name:
        layer_number = int(name.split('.')[2])
        if layer_number < model.config.n_layer - num_trainable_layers:
            param.requires_grad = False

# Display which layers are trainable
for name, param in model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")

gpt_neox.embed_in.weight: Trainable
gpt_neox.layers.0.input_layernorm.weight: Trainable
gpt_neox.layers.0.input_layernorm.bias: Trainable
gpt_neox.layers.0.post_attention_layernorm.weight: Trainable
gpt_neox.layers.0.post_attention_layernorm.bias: Trainable
gpt_neox.layers.0.attention.query_key_value.weight: Trainable
gpt_neox.layers.0.attention.query_key_value.bias: Trainable
gpt_neox.layers.0.attention.dense.weight: Trainable
gpt_neox.layers.0.attention.dense.bias: Trainable
gpt_neox.layers.0.mlp.dense_h_to_4h.weight: Trainable
gpt_neox.layers.0.mlp.dense_h_to_4h.bias: Trainable
gpt_neox.layers.0.mlp.dense_4h_to_h.weight: Trainable
gpt_neox.layers.0.mlp.dense_4h_to_h.bias: Trainable
gpt_neox.layers.1.input_layernorm.weight: Trainable
gpt_neox.layers.1.input_layernorm.bias: Trainable
gpt_neox.layers.1.post_attention_layernorm.weight: Trainable
gpt_neox.layers.1.post_attention_layernorm.bias: Trainable
gpt_neox.layers.1.attention.query_key_value.weight: Trainable
gpt_neox.layers.1.atte

* **Data Loader**: Batching is handled by torch.utils.data.DataLoader, which efficiently manages memory and computational overhead.
* **Validation Checks**: Periodically, the model's performance is evaluated on the validation set to monitor for overfitting and generalization.
* **Checkpoint Saving**: After each epoch, the model's state and optimizer's state are saved. This allows for recovery and fine-tuning from the last saved state in case the training is interrupted.

In [ ]:
from tqdm import tqdm

num_epochs = 2 # Set the number of epochs
model.train()  # Set the model to training mode

for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        # Move batch data to the same device as the model (GPU or CPU)
        inputs = {key: val.squeeze().to(device) for key, val in batch.items()}

        # Forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss

        # Zero gradients before backward pass
        optimizer.zero_grad()

        # Perform a backward pass to calculate the gradients
        loss.backward()

        # Update the weights
        optimizer.step()

        # Update learning rate schedule
        lr_scheduler.step()

        # Track the total loss for this epoch
        total_loss += loss.item()

    # Print average training loss per epoch
    print(f"Epoch {epoch+1}: Average training loss: {total_loss / len(train_loader)}")

    # Validation step after each training epoch
    model.eval()  # Set the model to evaluation mode
    total_eval_loss = 0
    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        for batch in tqdm(validation_loader, desc="Validating"):
            inputs = {key: val.squeeze().to(device) for key, val in batch.items()}
            outputs = model(**inputs, labels=inputs['input_ids'])
            loss = outputs.loss
            total_eval_loss += loss.item()

    # Calculate average loss over all validation batches
    avg_val_loss = total_eval_loss / len(validation_loader)
    print(f"Average validation loss: {avg_val_loss}")

    # Save model checkpoint after each epoch
    save_checkpoint(model, optimizer, filename=f"checkpoint_epoch_{epoch+1}.pth.tar")

    model.train()  # Reset to training mode for the next epoch

Epoch 1: 100%|██████████| 3500/3500 [28:15<00:00,  2.06it/s]


Epoch 1: Average training loss: 0.972862936769213


Validating: 100%|██████████| 750/750 [02:02<00:00,  6.11it/s]


Average validation loss: 1.0146829014619192


Epoch 2: 100%|██████████| 3500/3500 [28:07<00:00,  2.07it/s]


Epoch 2: Average training loss: 0.7135264207720756


Validating: 100%|██████████| 750/750 [02:01<00:00,  6.16it/s]


Average validation loss: 0.9765113667647044


## Model Evaluation

* Using BLEU and ROUGE scores

In [ ]:
!pip install nltk rouge-score
import nltk
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

nltk.download('punkt')

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=fae02c85e0e059790d6409acaaf1104ddab7e56c6862ecf2d310e899d7fc9b3a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Extracting Responses on the Test Dataset

In [ ]:
def generate_response(model, tokenizer, text, max_length=512):
    # Adjust the tokenizer setup to pad on the left (required for decoder-only models like GPT)
    tokenizer.padding_side = "left"

    # Encode the prompts to get tensor inputs
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length, add_special_tokens=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # It's essential to set both max_length and pad_token_id correctly
    adjusted_max_length = max_length + 50  # Allow room for the model to generate text beyond the length of the input

    # Generate response using the model
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=adjusted_max_length,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id  # Ensure padding token is set correctly for generation
    )

    # Decode the generated ids to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Ensure the model and tokenizer are on the same device
model.to(device)


# Function to apply generation to all test data
def generate_responses_for_test_data(model, tokenizer, test_data):
    responses = []
    for _, row in test_data.iterrows():
        prompt = row['message_1']  # Assuming the prompt is in 'message_1'
        response = generate_response(model, tokenizer, prompt)
        responses.append(response)
    return responses

# Generate responses
pythia_responses = generate_responses_for_test_data(model, tokenizer, test_data)

# Saving the generated responses as a csv file
test_data['generated_responses'] = pythia_responses
test_data.to_csv("test_with_responses.csv", index=False)


### Printing out evaluations from earlier

In [ ]:
from transformers import AutoTokenizer

# Assuming you're using the same tokenizer as for model inference
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m-v0")

def truncate_to_max_length(text, max_length=512):
    """ Truncate text to a maximum length in tokens. """
    tokens = tokenizer.tokenize(text)
    truncated_tokens = tokens[:max_length]  # Limit to max_length tokens
    return tokenizer.convert_tokens_to_string(truncated_tokens)

def evaluate_models(test_data, model_responses, reference_key='message_2', max_length=512):
    """ Evaluate model generated responses against truncated reference responses. """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    results = {}

    for model_name, responses in model_responses.items():
        print(f"Evaluating {model_name}...")
        references = [truncate_to_max_length(ref, max_length) for ref in test_data[reference_key].tolist()]
        truncated_responses = [truncate_to_max_length(resp, max_length) for resp in responses]

        bleu_score = corpus_bleu([[ref.split()] for ref in references], [resp.split() for resp in truncated_responses])
        rouge_scores = [scorer.score(ref, resp) for ref, resp in zip(references, truncated_responses)]

        avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
        avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

        results[model_name] = {
            'BLEU': bleu_score,
            'ROUGE-1': avg_rouge1,
            'ROUGE-L': avg_rougeL
        }

    return results


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(test_data.generated_responses)

5348     "How does the physiology of taste and smell wo...
339      Which specific domain would you assign to a sp...
13591    How have historical geological events affected...
8153     How can we develop more efficient methods for ...
16345    How does traumatic brain injury affect the neu...
                               ...                        
13191    What are the specific adaptations of savanna s...
3077     What evolutionary advantages are associated wi...
8393     How can biotechnology be utilized to develop m...
12187    How did the evolution of reptiles contribute t...
8962     "How does the physiological response of plants...
Name: generated_responses, Length: 3000, dtype: object


In [ ]:
test_data.head()

,role_1,topic;,sub_topic,message_1,message_2,generated_responses
5348,Biologist_RoleType.ASSISTANT,Physiology,The physiology of taste and smell,"""How does the physiology of taste and smell wo...",The physiology of taste and smell plays a cruc...,"""How does the physiology of taste and smell wo..."
339,Biologist_RoleType.ASSISTANT,Taxonomy,Identification of the correct domain for a giv...,Which specific domain would you assign to a sp...,The specific domain for a species of bacteria ...,Which specific domain would you assign to a sp...
13591,Biologist_RoleType.ASSISTANT,Biogeography,The biogeography of tropical ecosystems and th...,How have historical geological events affected...,Historical geological events have played a sig...,How have historical geological events affected...
8153,Biologist_RoleType.ASSISTANT,Biotechnology,Developing new methods for producing biofuels ...,How can we develop more efficient methods for ...,To develop more efficient methods for producin...,How can we develop more efficient methods for ...
16345,Biologist_RoleType.ASSISTANT,Neurobiology,The effects of traumatic brain injury,How does traumatic brain injury affect the neu...,Traumatic brain injury (TBI) can significantly...,How does traumatic brain injury affect the neu...


In [ ]:
model_responses = {
    'Pythia_LLM': pythia_responses
    #'RAG_Model': rag_responses
}

evaluation_results = evaluate_models(test_data, model_responses)
print(evaluation_results)


Evaluating Pythia_LLM...
{'Pythia_LLM': {'BLEU': 0.0026233595740149454, 'ROUGE-1': 0.2340466233453318, 'ROUGE-L': 0.16265453018815765}}


# Presentation and Final Evaluation

* [Presentation Video](https://youtu.be/OFHlgohwKRE)
* [Slides](https://docs.google.com/presentation/d/1JK7RCoR-Y1K8rXa9I6lHMnEE_9EcAIo4emaUtH05uqU/edit?usp=sharing)

* We conclude that overall the RAG seemed to perform better on this dataset than the SFT, it produced longer responses that were more accurate (according to BLEU and ROUGE-1 scores)
* Both models were limited by 512 tokens, and inference and GPU usage was a major issue
* Next Steps:
  * Implementing RAFT
  * Retraining entire model rather than just last two layers
